In [1]:
%load_ext autoreload
%autoreload 2

!jupyter lab clean
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension
!jupyter labextension install @jupyter-widgets/jupyterlab-manager
# !jupyter lab build

[LabCleanApp] Cleaning /opt/conda/share/jupyter/lab...
[LabCleanApp] Removing staging...
[LabCleanApp] Success!
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK
An error occurred.
PermissionError: [Errno 13] Permission denied: '/opt/conda/share/jupyter/lab/extensions/jupyter-widgets-jupyterlab-manager-3.0.1.tgz'
See the log file for details:  /tmp/jupyterlab-debug-p15nw1yc.log


In [2]:
%%html
<!-- Source: https://bl.ocks.org/texodus/96a9ed60d0250f7d3187c0fed5f5b78c -->
<style>
.brick {
    background-color: #c0c0c0; 
    border-top: 4px solid #fff;
    border-left: 4px solid #fff;
    border-bottom: 4px solid #808080;
    border-right: 4px solid #808080;
    font-size: 18px;
    font-weight: 900;
    font-family: 'Courier';

    padding: 0!important;
    margin: 0!important;
    border-radius: 0;
}

.oracle {
    background-color: #FF99CC!important;
}

.bomb-x {
    background-color: red!important;
}

.hint-0 {
    border: 1px solid #808080!important;
}
.hint-1 {
    color: #4200fe;
}
.hint-2 {
    color: #247f01;
}
.hint-3 {
    color: #ef1c03;
}
.hint-4 {
    color: #1d0181;
}
.hint-5 {
    color: #810a02;
}
.hint-6 {
    color: #308081;
}
.hint-7 {
    color: #000000;
}
.hint-8 {
    color: #808080;
}

.widget-label {
    white-space: normal!important;
    line-height: 1.2;
}
</style>

In [3]:
import copy
import itertools
from functools import partial

from IPython.display import display
from ipywidgets import Button, GridspecLayout, Layout, ButtonStyle, GridBox, Box, Image, Label
import ipywidgets

from solver import QAOAMinesweeperSolver


class GUI:
    
    def __init__(self, n_rows, n_cols, solution):
        self.n_rows = n_rows
        self.n_cols = n_cols

        self.solution = solution
        self.board_state = [['.'] * n_cols for _ in range(n_rows)]
        
        self.game_over = False
        self.cells = [[None] * n_cols for _ in range(n_rows)]
        

    def draw(self):
        for r in range(self.n_rows):
            for c in range(self.n_cols):
                button = Button(
                    layout=Layout(width='auto', height='auto'), 
                    style=ButtonStyle(button_color='#c3c3c3'),
                )
                
                button.on_click(partial(self.button_clicked, r, c))
                button.add_class('brick')
                self.cells[r][c] = button
        
        size = 500 // min(self.n_rows, self.n_cols)
        row_template = ' '.join([f'{size}px'] * self.n_rows)
        col_template = ' '.join([f'{size}px'] * self.n_cols)
        
        game_grid = GridBox(
            children=list(itertools.chain(*self.cells)),
            layout=Layout(
                grid_template_rows=row_template,
                grid_template_columns=col_template,
                margin='1em'
            )
        )
        
        reset_button=Button(description='Reset', style=ButtonStyle(button_color='#f2e237'))
        help_button=Button(description='Quantum Help', style=ButtonStyle(button_color='#FF99CC'))
        
        reset_button.on_click(lambda _: self.reset_game())
        help_button.on_click(lambda _: self.get_help())
        
        
        self.quantum_oracle_label = Label(value="I'd love to assist you using my quantum-magical powers.")
        
        with open("img/quantum_oracle.png", "rb") as file:
            image = file.read()
            self.quantum_oracle = Image(
                value=image, 
                format='png',
                layout=Layout(height='400px')
            )
        
        side_box = Box(
            children=[self.quantum_oracle_label, self.quantum_oracle, help_button, reset_button],
            layout=Layout(display='flex',
                    flex_flow='column',
                    align_items='center',
                    width='300px',
                    margin='1em')
        )
                
        window = Box(
            children=[game_grid, side_box],
            layout=Layout(display='flex',
                    flex_flow='row',
                    align_items='center')
        )
        
        return window
    
    def button_clicked(self, r, c, button):
        if self.game_over: return
        
        # Is it a Bomb?
        self.game_over = self.solution[r][c] == 'x'
        
        self.reveal(r, c)
        self.update_ui(r, c)
        
        # Check if all bombs revealed
        n_bombs = 0
        n_spaces = 0
        for r in range(self.n_rows):
            for c in range(self.n_cols):
                n_bombs  += 1 if self.solution[r][c] == 'x' else 0
                n_spaces += 1 if self.board_state[r][c] == '.' else 0
                
        if n_bombs == n_spaces:
            self.quantum_oracle_label.value = "Congratulations! You won!\nDid you know, that when you play Minesweeper with your eyes closed, you are winning and loosing at the same time? At least until you open your eyes."
            with open("img/quantum_oracle_win.png", "rb") as file:
                self.quantum_oracle.value = file.read()
                
            
    def reveal(self, r, c):
        if self.board_state[r][c] != '.':
            return  # Already revealed
        
        self.board_state[r][c] = self.solution[r][c]
        if self.solution[r][c] != '0':
            return  # Stop exploring
        
        for (_r, _c) in itertools.product([r-1,r,r+1], [c-1,c,c+1]):
                if (0 <= _r < self.n_rows and 0 <= _c < self.n_cols) and self.board_state[_r][_c] == '.': 
                    self.reveal(_r, _c)
                
    def update_ui(self, btn_r, btn_c):
        for r in range(self.n_rows):
            for c in range(self.n_cols):
                button = self.cells[r][c]
                state = self.board_state[r][c]
                
                if state.isdigit():
                    button.description = ' ' if state == '0' else state
                    button.add_class('hint-' + state)
                    button.remove_class('oracle')
                else:
                    button.description = ' '
        
        if self.game_over:
            for r in range(self.n_rows):
                for c in range(self.n_cols):
                    button = self.cells[r][c]
                    if self.solution[r][c] == 'x':
                        button.description = '💣'
                        button.add_class('bomb')
            
            self.cells[btn_r][btn_c].add_class('bomb-x')

                    
    def reset_game(self):
        self.game_over = False
        self.board_state = [['.'] * self.n_cols for _ in range(self.n_rows)]
        
        for cls in (['hint-' + str(i) for i in range(9)] + ['bomb', 'bomb-x', 'oracle']):
            for cell in itertools.chain(*self.cells):
                cell.remove_class(cls)
                
        self.update_ui(None, None)
    
    def get_help(self):
        print('HELP!')
        
        self.quantum_oracle_label.value = "Let me think for a moment..."
        
        board = copy.deepcopy(self.board_state)
        
        for r in range(self.n_rows):
            for c in range(self.n_cols):
                for (_r, _c) in itertools.product([r-1,r,r+1], [c-1,c,c+1]):
                    if not (0 <= _r < self.n_rows and 0 <= _c < self.n_cols): 
                        continue
                    if self.board_state[_r][_c] != '.':
                        break
                else:
                    board[r][c] = '-'

        qc_board = '\n'.join(map(lambda e: ''.join(e), board))
        solution, cost = QAOAMinesweeperSolver().solve(qc_board)
        
        if cost == 0:
            self.quantum_oracle_label.value = "I found a solution! But watch out, I can't guarantee that it is unique."
        else:
            self.quantum_oracle_label.value = "Hmm... This is a lot harder than I thought. I found a possible solution, but I'm not too confident in it."
        
        print(f'Found solution {solution} with cost {cost}')
        self.display_oracle(solution, board)

    def display_oracle(self, prediction, qc_board):
        p_index = 0
        
        for r in range(self.n_rows):
            for c in range(self.n_cols):
                self.cells[r][c].remove_class('oracle')
                if qc_board[r][c] != '.': continue

                if prediction[p_index] == '1':
                    self.cells[r][c].add_class('oracle')
                
                p_index += 1
        

In [6]:
from minesweeper import generate_random_board

board = generate_random_board(12, 12, 12).split('\n')

gui = GUI(len(board), len(board[0]), board)
gui.draw()

Box(children=(GridBox(children=(Button(layout=Layout(height='auto', width='auto'), style=ButtonStyle(button_co…